# Split Sample in Time Series

## Skills: Data Cleaning, Calculating RFM features, Statistical Features, Splitting Sample According to Different Time Frames

In machine learning, it is necessary to split sample into train and test sets in order to evaluate model performance.  In classification problems, the distribution of target variable should be similar in test and train sets.  In case of time series problems, training data should be extracted from an earlier time frame than testing data. The following example uses transaction data from a retail bank to predict customer churn.  It shows the process of using transaction data to calculate recency, frequency and monetary value features.  In addition, statistical features such as mean, standard deviation, minimum, maximum, skewness and kurtosis of the transaction records are calculated.  Lastly, it shows how to split the sample into train and test sets, with full consideration of different time frames and target variable distribution.

The data is split into 2 time frames: training period is from Feb 1, 2018 to Jan 30, 2019.  Training labels were genenrated from Feb 1 to Apr 30, 2019.  The testing period is from May 1, 2018 to Apr 30, 2019.  The testing labels are generated from the data in May 1, 2019 to July 31, 2019.  Each period lasts for 12 months.  Customers with at least 6 months of transaction record are included in the analysis.

In [1]:
import numpy as np
import pandas as pd
from numpy import unique
from pandas import read_csv
import datetime as dt
import matplotlib.pyplot as plt

In [2]:
#Load customer profile data
S6a = read_csv('/Users/hleung/PycharmProjects/S6a.csv', header=0)
S6a.shape

(34812, 11)

The dataset has 34812 customer records with 11 features.

In [4]:
S6a.head(5)

,Memberid,Status,OpenDate,CloseDate,Age,NumberofSavingsProducts,CountofLoans,TenureMonth,DBINDICATOR,CCINDICATOR,INDIRECT
0,4444R4MGG,Open,2017-08-18,NaN,80.0,1.0,1,23,0,0,1
1,444F4RR4K,Open,2017-11-17,NaN,31.0,1.0,1,20,0,0,0
2,444G55TMG,Open,2016-12-08,NaN,20.0,2.0,0,31,1,0,0
3,444RMTMGF,Open,2018-02-15,NaN,31.0,1.0,1,17,0,0,1
4,444ZZ88RM,Open,2018-10-19,NaN,62.0,2.0,0,9,1,0,0


In [5]:
# Feature Description:
# Memberid: A unique customer id
# Status: Open / Closed depends on whether th customer has closed the account or not
# OpenDate, Close Date: The time when the customer open or close the account
# Age: Customer age
# NumberofSavingsProducts: No. of savings products the customer is having in the bank
# CountofLoans: No. of Loans the customer is having in the bank
# TenureMonth: The length of time in months the customer is with the bank
# DBINDICATOR: Whether the customer has a debit card with the bank (1 or 0)
# CCINDICATOR: Whether the customer has a credit card with the bank (1 or 0)
# INDIRECT: Whether the customer has an indirect loan with the bank (1 or 0)

## Feature Engineering for Training Period

### Generating Target Label in Train Set

Using customer records during the training time frame, target label churn is generated based on the status between Feb 1, 2019 and May 1, 2019.

In [3]:
S6a['CloseDate'] = pd.to_datetime(S6a['CloseDate'])
S6a['OpenDate'] = pd.to_datetime(S6a['OpenDate'])
cutoff1 = dt.datetime(2018, 8, 1) #6 months before last day of train data
cutoff2 = dt.datetime(2019, 2, 1) #first day of train label
cutoff3 = dt.datetime(2019, 5, 1) #last day of train label
Open = S6a[(S6a['OpenDate'] < cutoff1) & (S6a['Status'] == 'Open')] #extract accounts at are active during
#the training period and have at least 6 months transaction record.
#Extract accounts that are active during training period but closed between Feb and Apr 2019
Closed = S6a[(S6a['OpenDate'] < cutoff1) & (S6a['CloseDate'] > cutoff2) & (S6a['CloseDate'] < cutoff3)] #666 rows
#Stack and sort the dataframes
train6 = pd.concat([Open, Closed], ignore_index=True)
train6 = train6.sort_values('Memberid')
train6['Status'].value_counts()

Open      23963
Closed      666
Name: Status, dtype: int64

There are a total of 24629 customers active during the training period, 2.7% churn of them would churn within 3 months afterwards.

### Extract Transaction Records for Train Set

This transaction dataset contains all monthly transaction records. The number of rows each customer has depnds on the number of months he/she is with the bank. The first row the start date and the last row is the close date (if the customer churns) or the cutoff date when the dataset was created (if the account is open).

In [4]:
S7 = read_csv('/Users/hleung/PycharmProjects/S7.csv', header=0)
S7.shape

(671562, 19)

In [11]:
S7.head(20)

,Memberid,Date,Status,Age,TenureMonth,NoOfSavingsP,NoOfLoans,NumberofDirectDeposits,SumofDirectDeposits,NumberofBillPayTransactions,NumberofDebitCardTransactions,NumberofTransactionsConductedinBranch,FEESCHARGED,SUMofFEESCHARGED,NumberofTotalTransactions,OnlineTran,NumberofPayments,SumofPaymentAmount,AllTran
0,584488448,2018-01-31,0,67.0,1,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
1,584488448,2018-02-28,0,67.0,2,1.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0
2,584488448,2018-03-31,0,67.0,3,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
3,584488448,2018-04-30,0,67.0,4,1.0,0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,NaN,NaN,NaN,2.0
4,584488448,2018-05-31,0,67.0,5,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
5,584488448,2018-06-30,0,67.0,6,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
6,584488448,2018-07-31,0,67.0,7,1.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0
7,584488448,2018-08-31,0,67.0,8,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
8,584488448,2018-09-30,0,67.0,9,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0
9,584488448,2018-10-31,0,67.0,10,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0


There are 671562 customer records.  Each row shows the monthly transaction for one customer.  There are 19 features.  The following is the description of selected features.

In [5]:
#Feature Description (selected)
#Status: 0-active, 1-the month before churn, 2-churn
#FEESCHARGED - No. of times service fee is charged
#Online Tran - No. of online transactions
#NumberofPayments - No. of loan payments
#SumofPaymentAmount - Sum of loan payments
#AllTran - Total no. of transactions in that month

In [6]:
#Extract transaction records during training period.
cutoff4 = dt.datetime(2018, 2, 1) #First day of training period
namelist = train6['Memberid']
Tran = S7[S7['Memberid'].isin(namelist) == True]
Tran['Date'] = pd.to_datetime(Tran['Date'])
Tran = Tran[(Tran['Date'] >= cutoff4) & (Tran['Date'] < cutoff2)] 

/Users/hleung/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [14]:
Tran.shape

(270396, 19)

### Extract Tenure Month in Train Set

In [7]:
#Extract the tenure month when training period ends
TM = Tran[['Memberid', 'TenureMonth']]
TM = TM.groupby(['Memberid']).tail(1) #extract the last TenureMonth in each customer.
TM = TM.rename({'TenureMonth':'TM'}, axis=1)
train6 = pd.merge(train6, TM, on=['Memberid'], how='left') #merge to the profile dataset

### Recency, Frequency and Monetary Value (RFM) Features in Train Set 

In [8]:
#Calculate Recency in train set (the time gap of the most recent transaction)
Tran = Tran.fillna(0)
Tran1 = Tran[Tran['AllTran'] != 0] #extract all non-zero transactions
Tran1 = Tran1.groupby(['Memberid']).tail(1) #extract the last non-zero transaction
Tran1 = Tran1[['Memberid', 'Date']]
Tran1 = Tran1.rename({'Date':'MostRecent'}, axis=1) #rename it as most recent transaction
train6 = pd.merge(train6, Tran1, on=['Memberid'], how='left')
train6.isnull().sum() #1744 missing rows in MostRecent because those members have no activities during 
#  training period, replace with OpenDate
train6.MostRecent = np.where(train6.MostRecent.isnull(), train6.OpenDate, train6.MostRecent)
train6['MostRecent'] = pd.to_datetime(train6['MostRecent'])
#time gap function
def months(d1, d2):
    return d1.month - d2.month + 12*(d1.year - d2.year)
train6['RecencyGap'] = train6.apply(lambda x: months(cutoff2, x.MostRecent), axis=1) #Time gap between 
#last day of training period and the most recent transaction.

In [9]:
#Calculating Frequency in train set (average no. of transactions per month)
Tran1 = Tran[['Memberid', 'AllTran']]
Tran1 = Tran1.groupby(['Memberid']).mean()
Tran1 = Tran1.rename({'AllTran':'Freq'}, axis=1)
train6 = pd.merge(train6, Tran1, on=['Memberid'], how='left')

In [10]:
#Monetary value: Direct Deposit & Loan Payment Amount
Tran['SumofPaymentAmount'] = Tran['SumofPaymentAmount']*-1 #Turn negative values to positive
Tran1 = Tran[['Memberid', 'SumofDirectDeposits', 'SumofPaymentAmount']]
Tran1 = Tran1.groupby(['Memberid']).mean()
Tran1 = Tran1.rename({'SumofDirectDeposits':'DDAmt', 'SumofPaymentAmount':'LoanPayAmt'}, axis=1)
Tran1.isin([0]).sum() # >50% is 0
train6 = pd.merge(train6, Tran1, on=['Memberid'], how='left')

### Statistical Features

Besides RFM features, statistical features like mean, standard deviation, maximum, minimum, skewness and kurtosis of each time series are calculated and merged to customer profile dataset.

In [11]:
#Excluding columns that are already in the RFM features
Tran1 = Tran[['Memberid', 'NumberofDirectDeposits', 'NumberofBillPayTransactions',
              'NumberofDebitCardTransactions', 'NumberofTransactionsConductedinBranch', 'FEESCHARGED', 'SUMofFEESCHARGED',
              'NumberofTotalTransactions', 'OnlineTran', 'NumberofPayments']]
Tran2 = Tran1.groupby(['Memberid']).mean()
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_mean', 'NumberofBillPayTransactions':'NofBP_mean',
                      'NumberofDebitCardTransactions': 'NofDC_mean', 'NumberofTransactionsConductedinBranch': 'NofIB_mean',
                      'FEESCHARGED': 'FC_mean', 'SUMofFEESCHARGED': 'SofFC_mean', 'NumberofTotalTransactions': 'NofTT_mean',
                      'OnlineTran':'OL_mean', 'NumberofPayments': 'NofLP_mean'}, axis=1)
train6 = pd.merge(train6, Tran2, on=['Memberid'], how='left')

In [12]:
#Calculate SD of all transaction data, including RFM factors in train set
Tran1 = Tran[['Memberid', 'NumberofDirectDeposits', 'SumofDirectDeposits', 'NumberofBillPayTransactions',
              'NumberofDebitCardTransactions', 'NumberofTransactionsConductedinBranch', 'FEESCHARGED', 'SUMofFEESCHARGED',
              'NumberofTotalTransactions', 'OnlineTran', 'NumberofPayments', 'SumofPaymentAmount', 'AllTran']]
Tran2 = Tran1.groupby('Memberid').std(ddof=0)
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_sd', 'SumofDirectDeposits': 'SofDD_sd', 'NumberofBillPayTransactions':'NofBP_sd',
                      'NumberofDebitCardTransactions': 'NofDC_sd', 'NumberofTransactionsConductedinBranch': 'NofIB_sd',
                      'FEESCHARGED': 'FC_sd', 'SUMofFEESCHARGED': 'SofFC_sd', 'NumberofTotalTransactions': 'NofTT_sd',
                      'OnlineTran':'OL_sd', 'NumberofPayments': 'NofLP_sd', 'SumofPaymentAmount': 'SofPA_sd,',
                      'AllTran': 'AllTran_sd'}, axis=1)
train6 = pd.merge(train6, Tran2, on=['Memberid'], how='left')

In [13]:
#Maximum in train set
Tran2 = Tran1.groupby('Memberid').max()
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_max', 'SumofDirectDeposits': 'SofDD_max', 'NumberofBillPayTransactions':'NofBP_max',
                      'NumberofDebitCardTransactions': 'NofDC_max', 'NumberofTransactionsConductedinBranch': 'NofIB_max',
                      'FEESCHARGED': 'FC_max', 'SUMofFEESCHARGED': 'SofFC_max', 'NumberofTotalTransactions': 'NofTT_max',
                      'OnlineTran':'OL_max', 'NumberofPayments': 'NofLP_max', 'SumofPaymentAmount': 'SofPA_max,',
                      'AllTran': 'AllTran_max'}, axis=1)
train6 = pd.merge(train6, Tran2, on=['Memberid'], how='left')

In [14]:
#Minimum in train set
Tran2 = Tran1.groupby('Memberid').min()
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_min', 'SumofDirectDeposits': 'SofDD_min', 'NumberofBillPayTransactions':'NofBP_min',
                      'NumberofDebitCardTransactions': 'NofDC_min', 'NumberofTransactionsConductedinBranch': 'NofIB_min',
                      'FEESCHARGED': 'FC_min', 'SUMofFEESCHARGED': 'SofFC_min', 'NumberofTotalTransactions': 'NofTT_min',
                      'OnlineTran':'OL_min', 'NumberofPayments': 'NofLP_min', 'SumofPaymentAmount': 'SofPA_min,',
                      'AllTran': 'AllTran_min'}, axis=1)
train6 = pd.merge(train6, Tran2, on=['Memberid'], how='left')

In [15]:
#Skewness in train set
Tran2 = Tran1.groupby('Memberid').skew()
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_sk', 'SumofDirectDeposits': 'SofDD_sk', 'NumberofBillPayTransactions':'NofBP_sk',
                      'NumberofDebitCardTransactions': 'NofDC_sk', 'NumberofTransactionsConductedinBranch': 'NofIB_sk',
                      'FEESCHARGED': 'FC_sk', 'SUMofFEESCHARGED': 'SofFC_sk', 'NumberofTotalTransactions': 'NofTT_sk',
                      'OnlineTran':'OL_sk', 'NumberofPayments': 'NofLP_sk', 'SumofPaymentAmount': 'SofPA_sk,',
                      'AllTran': 'AllTran_sk'}, axis=1)
train6= pd.merge(train6, Tran2, on=['Memberid'], how='left')

In [16]:
#Kurtosis in train set
Tran1 = Tran1.fillna(0)
Tran1 = Tran1.set_index('Memberid')
Tran2 = Tran1.groupby('Memberid').apply(pd.DataFrame.kurt)
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_k', 'SumofDirectDeposits': 'SofDD_k', 'NumberofBillPayTransactions':'NofBP_k',
                      'NumberofDebitCardTransactions': 'NofDC_k', 'NumberofTransactionsConductedinBranch': 'NofIB_k',
                      'FEESCHARGED': 'FC_k', 'SUMofFEESCHARGED': 'SofFC_k', 'NumberofTotalTransactions': 'NofTT_k',
                      'OnlineTran':'OL_k', 'NumberofPayments': 'NofLP_k', 'SumofPaymentAmount': 'SofPA_k,',
                      'AllTran': 'AllTran_k'}, axis=1)
train6 = pd.merge(train6, Tran2, on=['Memberid'], how='left')

In [17]:
#Convert Target to dummy variable in train set
Label = train6[['Memberid', 'Status']]
dummy = pd.get_dummies(Label, columns=["Status"])
dummy = dummy.drop(['Status_Open'], axis = 1)
train6 = pd.merge(train6, dummy, on=['Memberid'], how='left')

In [18]:
train6.head(5)

,Memberid,Status,OpenDate,CloseDate,Age,NumberofSavingsProducts,CountofLoans,TenureMonth,DBINDICATOR,CCINDICATOR,...,NofDC_k,NofIB_k,FC_k,SofFC_k,NofTT_k,OL_k,NofLP_k,"SofPA_k,",AllTran_k,Status_Closed
0,4444R4MGG,Open,2017-08-18,NaT,80.0,1.0,1,23,0,0,...,0.000000,12.000000,12.0,12.0,2.640000,8.884615,-1.260793,-1.409994,1.761862,0
1,444F4RR4K,Open,2017-11-17,NaT,31.0,1.0,1,20,0,0,...,0.000000,3.165306,0.0,0.0,4.119611,1.701563,5.500000,5.528658,1.328970,0
2,444G55TMG,Open,2016-12-08,NaT,20.0,2.0,0,31,1,0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,444RMTMGF,Open,2018-02-15,NaT,31.0,1.0,1,17,0,0,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.069547,0.000000,0
4,445445MRK,Open,2018-07-10,NaT,77.0,2.0,0,12,1,0,...,4.639906,6.000000,0.0,0.0,2.774400,0.000000,0.000000,0.000000,2.774400,0


## Feature Engineering for Testing Period

The testing period is from May 1 2018 to Apr 30, 2019.  All customers who are active and have at least 6 months of transaction record are extracted.  RFM and statistical features are calculated for each time series.

In [4]:
S6a['CloseDate'] = pd.to_datetime(S6a['CloseDate'])
S6a['OpenDate'] = pd.to_datetime(S6a['OpenDate'])
cutoff1 = dt.datetime(2018, 11, 1) #6 months before last day of test data
cutoff2 = dt.datetime(2019, 5, 1) #first day of test label
cutoff3 = dt.datetime(2019, 8, 1) #last day of test label
Open = S6a[(S6a['OpenDate'] < cutoff1) & (S6a['Status'] == 'Open')] #extract accounts at are active during
#the testing period and have at least 6 months transaction record.
#Extract accounts that are active during testing period but closed between May and July 2019
Closed = S6a[(S6a['OpenDate'] < cutoff1) & (S6a['CloseDate'] > cutoff2) & (S6a['CloseDate'] < cutoff3)]
#Stack and sort the dataframes
test6 = pd.concat([Open, Closed], ignore_index=True)
test6 = test6.sort_values('Memberid')
test6['Status'].value_counts()

Open      27881
Closed      841
Name: Status, dtype: int64

There are 28722 customers in testing period, 3% of them churn within 3 months afterwards.

### Extract Transaction Records for Test Set

In [ ]:
#Extract transaction records during testing period.
cutoff4 = dt.datetime(2018, 5, 1) #First day of testing period
namelist = test6['Memberid']
Tran = S7[S7['Memberid'].isin(namelist) == True]
Tran['Date'] = pd.to_datetime(Tran['Date'])
Tran = Tran[(Tran['Date'] >= cutoff4) & (Tran['Date'] < cutoff2)] 

### Extract Tenure Month in Test Set

In [8]:
#Extract the tenure month when testing period ends
TM = Tran[['Memberid', 'TenureMonth']]
TM = TM.groupby(['Memberid']).tail(1) #extract the last TenureMonth in each customer.
TM = TM.rename({'TenureMonth':'TM'}, axis=1)
test6 = pd.merge(test6, TM, on=['Memberid'], how='left') #merge to the profile dataset

### Recency, Frequency and Monetary Value (RFM) Features in Test Set 

In [9]:
#Calculate Recency in test set (the time gap of the most recent transaction)
Tran = Tran.fillna(0)
Tran1 = Tran[Tran['AllTran'] != 0] #extract all non-zero transactions
Tran1 = Tran1.groupby(['Memberid']).tail(1) #extract the last non-zero transaction
Tran1 = Tran1[['Memberid', 'Date']]
Tran1 = Tran1.rename({'Date':'MostRecent'}, axis=1) #rename it as most recent transaction
test6 = pd.merge(test6, Tran1, on=['Memberid'], how='left')
#Replace missing rows with OpenDate
test6.MostRecent = np.where(test6.MostRecent.isnull(), test6.OpenDate, test6.MostRecent)
test6['MostRecent'] = pd.to_datetime(test6['MostRecent'])
#time gap function
def months(d1, d2):
    return d1.month - d2.month + 12*(d1.year - d2.year)
test6['RecencyGap'] = test6.apply(lambda x: months(cutoff2, x.MostRecent), axis=1) #time gap between last
#day of testing period and the most recent transaction.

In [10]:
#Calculating Frequency in test set (average no. of transactions per month)
Tran1 = Tran[['Memberid', 'AllTran']]
Tran1 = Tran1.groupby(['Memberid']).mean()
Tran1 = Tran1.rename({'AllTran':'Freq'}, axis=1)
test6 = pd.merge(test6, Tran1, on=['Memberid'], how='left')

In [11]:
#Monetary value: Direct Deposit & Loan Payment Amount in test set
Tran['SumofPaymentAmount'] = Tran['SumofPaymentAmount']*-1 #Turn negative values to positive
Tran1 = Tran[['Memberid', 'SumofDirectDeposits', 'SumofPaymentAmount']]
Tran1 = Tran1.groupby(['Memberid']).mean()
Tran1 = Tran1.rename({'SumofDirectDeposits':'DDAmt', 'SumofPaymentAmount':'LoanPayAmt'}, axis=1)
Tran1.isin([0]).sum() # >50% is 0
test6 = pd.merge(test6, Tran1, on=['Memberid'], how='left')

### Statistical Features in Test Set

In [12]:
#Excluding columns that are already in the RFM features
Tran1 = Tran[['Memberid', 'NumberofDirectDeposits', 'NumberofBillPayTransactions',
              'NumberofDebitCardTransactions', 'NumberofTransactionsConductedinBranch', 'FEESCHARGED', 'SUMofFEESCHARGED',
              'NumberofTotalTransactions', 'OnlineTran', 'NumberofPayments']]
Tran2 = Tran1.groupby(['Memberid']).mean()
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_mean', 'NumberofBillPayTransactions':'NofBP_mean',
                      'NumberofDebitCardTransactions': 'NofDC_mean', 'NumberofTransactionsConductedinBranch': 'NofIB_mean',
                      'FEESCHARGED': 'FC_mean', 'SUMofFEESCHARGED': 'SofFC_mean', 'NumberofTotalTransactions': 'NofTT_mean',
                      'OnlineTran':'OL_mean', 'NumberofPayments': 'NofLP_mean'}, axis=1)
test6 = pd.merge(test6, Tran2, on=['Memberid'], how='left')

In [13]:
#Calculate SD of all transaction data, including RFM factors in test set
Tran1 = Tran[['Memberid', 'NumberofDirectDeposits', 'SumofDirectDeposits', 'NumberofBillPayTransactions',
              'NumberofDebitCardTransactions', 'NumberofTransactionsConductedinBranch', 'FEESCHARGED', 'SUMofFEESCHARGED',
              'NumberofTotalTransactions', 'OnlineTran', 'NumberofPayments', 'SumofPaymentAmount', 'AllTran']]
Tran2 = Tran1.groupby('Memberid').std(ddof=0)
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_sd', 'SumofDirectDeposits': 'SofDD_sd', 'NumberofBillPayTransactions':'NofBP_sd',
                      'NumberofDebitCardTransactions': 'NofDC_sd', 'NumberofTransactionsConductedinBranch': 'NofIB_sd',
                      'FEESCHARGED': 'FC_sd', 'SUMofFEESCHARGED': 'SofFC_sd', 'NumberofTotalTransactions': 'NofTT_sd',
                      'OnlineTran':'OL_sd', 'NumberofPayments': 'NofLP_sd', 'SumofPaymentAmount': 'SofPA_sd,',
                      'AllTran': 'AllTran_sd'}, axis=1)
test6 = pd.merge(test6, Tran2, on=['Memberid'], how='left')

In [14]:
#Maximum in test set
Tran2 = Tran1.groupby('Memberid').max()
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_max', 'SumofDirectDeposits': 'SofDD_max', 'NumberofBillPayTransactions':'NofBP_max',
                      'NumberofDebitCardTransactions': 'NofDC_max', 'NumberofTransactionsConductedinBranch': 'NofIB_max',
                      'FEESCHARGED': 'FC_max', 'SUMofFEESCHARGED': 'SofFC_max', 'NumberofTotalTransactions': 'NofTT_max',
                      'OnlineTran':'OL_max', 'NumberofPayments': 'NofLP_max', 'SumofPaymentAmount': 'SofPA_max,',
                      'AllTran': 'AllTran_max'}, axis=1)
test6 = pd.merge(test6, Tran2, on=['Memberid'], how='left')

In [15]:
#Minimum in test set
Tran2 = Tran1.groupby('Memberid').min()
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_min', 'SumofDirectDeposits': 'SofDD_min', 'NumberofBillPayTransactions':'NofBP_min',
                      'NumberofDebitCardTransactions': 'NofDC_min', 'NumberofTransactionsConductedinBranch': 'NofIB_min',
                      'FEESCHARGED': 'FC_min', 'SUMofFEESCHARGED': 'SofFC_min', 'NumberofTotalTransactions': 'NofTT_min',
                      'OnlineTran':'OL_min', 'NumberofPayments': 'NofLP_min', 'SumofPaymentAmount': 'SofPA_min,',
                      'AllTran': 'AllTran_min'}, axis=1)
test6 = pd.merge(test6, Tran2, on=['Memberid'], how='left')

In [16]:
#Skewness in test set
Tran2 = Tran1.groupby('Memberid').skew()
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_sk', 'SumofDirectDeposits': 'SofDD_sk', 'NumberofBillPayTransactions':'NofBP_sk',
                      'NumberofDebitCardTransactions': 'NofDC_sk', 'NumberofTransactionsConductedinBranch': 'NofIB_sk',
                      'FEESCHARGED': 'FC_sk', 'SUMofFEESCHARGED': 'SofFC_sk', 'NumberofTotalTransactions': 'NofTT_sk',
                      'OnlineTran':'OL_sk', 'NumberofPayments': 'NofLP_sk', 'SumofPaymentAmount': 'SofPA_sk,',
                      'AllTran': 'AllTran_sk'}, axis=1)
test6= pd.merge(test6, Tran2, on=['Memberid'], how='left')

In [17]:
#Kurtosis in train set
Tran1 = Tran1.fillna(0)
Tran1 = Tran1.set_index('Memberid')
Tran2 = Tran1.groupby('Memberid').apply(pd.DataFrame.kurt)
Tran2 = Tran2.rename({'NumberofDirectDeposits':'NofDD_k', 'SumofDirectDeposits': 'SofDD_k', 'NumberofBillPayTransactions':'NofBP_k',
                      'NumberofDebitCardTransactions': 'NofDC_k', 'NumberofTransactionsConductedinBranch': 'NofIB_k',
                      'FEESCHARGED': 'FC_k', 'SUMofFEESCHARGED': 'SofFC_k', 'NumberofTotalTransactions': 'NofTT_k',
                      'OnlineTran':'OL_k', 'NumberofPayments': 'NofLP_k', 'SumofPaymentAmount': 'SofPA_k,',
                      'AllTran': 'AllTran_k'}, axis=1)
test6 = pd.merge(test6, Tran2, on=['Memberid'], how='left')

In [18]:
#Convert Target to dummy variable in train set
Label = test6[['Memberid', 'Status']]
dummy = pd.get_dummies(Label, columns=["Status"])
dummy = dummy.drop(['Status_Open'], axis = 1)
test6 = pd.merge(test6, dummy, on=['Memberid'], how='left')

In [19]:
test6.head(5)

,Memberid,Status,OpenDate,CloseDate,Age,NumberofSavingsProducts,CountofLoans,TenureMonth,DBINDICATOR,CCINDICATOR,...,NofDC_k,NofIB_k,FC_k,SofFC_k,NofTT_k,OL_k,NofLP_k,"SofPA_k,",AllTran_k,Status_Closed
0,4444R4MGG,Open,2017-08-18,NaT,80.0,1.0,1,23,0,0,...,0.0,2.640000,2.640000,2.640000,-0.325926,6.767753,-0.855556,-0.939672,1.541522,0
1,444F4RR4K,Open,2017-11-17,NaT,31.0,1.0,1,20,0,0,...,0.0,3.165306,0.000000,0.000000,4.441695,2.226024,0.000000,-1.325904,1.731529,0
2,444G55TMG,Open,2016-12-08,NaT,20.0,2.0,0,31,1,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,444RMTMGF,Open,2018-02-15,NaT,31.0,1.0,1,17,0,0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.276490,0.000000,0
4,444ZZ88RM,Open,2018-10-19,NaT,62.0,2.0,0,9,1,0,...,0.0,0.000000,-3.333333,-3.333333,-2.806953,0.000000,0.000000,0.000000,-2.806953,0


## Creating Train and Test Sets

Since the training period and testing period overlap, there are many customers appear in both periods.  When creating the train and test sets, non-overlapping customers will be extracted first and included in their corresponding train or test sets.  The overlapping customers, mostly active ones, will be split to 75% to 25% ratio (train: test).  Therefore, no customer appears both in train and test sets.

In [19]:
test6 = read_csv('test6.csv', header=0)

In [21]:
train6close = train6[train6['Status'] == 'Closed']
train6open = train6[train6['Status'] == 'Open']
namelist = train6open['Memberid']
Test = test6[test6['Memberid'].isin(namelist) == False]
Xopen = train6open[['Memberid', 'Age', 'NumberofSavingsProducts', 'CountofLoans', 'DBINDICATOR', 'CCINDICATOR', 'INDIRECT',
               'RecencyGap', 'Freq', 'DDAmt', 'LoanPayAmt', 'TM', 'NofDD_mean', 'NofBP_mean', 'NofDC_mean', 'NofIB_mean',
               'FC_mean', 'SofFC_mean', 'NofTT_mean', 'OL_mean', 'NofLP_mean', 'NofDD_sd', 'NofBP_sd', 'NofDC_sd', 'NofIB_sd',
               'FC_sd', 'SofFC_sd', 'NofTT_sd', 'OL_sd', 'NofLP_sd', 'NofDD_max', 'NofBP_max', 'NofDC_max', 'NofIB_max',
               'FC_max', 'SofFC_max', 'NofTT_max', 'OL_max', 'NofLP_max', 'NofDD_min', 'NofBP_min', 'NofDC_min', 'NofIB_min',
               'FC_min', 'SofFC_min', 'NofTT_min', 'OL_min', 'NofLP_min', 'NofDD_sk', 'NofBP_sk', 'NofDC_sk', 'NofIB_sk',
               'FC_sk', 'SofFC_sk', 'NofTT_sk', 'OL_sk', 'NofLP_sk', 'NofDD_k', 'NofBP_k', 'NofDC_k', 'NofIB_k',
               'FC_k', 'SofFC_k', 'NofTT_k', 'OL_k', 'NofLP_k']] #Extract input features for open cases
yopen = train6open[['Memberid', 'Status_Closed']]

In [22]:
from sklearn.model_selection import train_test_split

# split into open cases to train and test sets
X_train, X_test, y_train, y_test = train_test_split(Xopen, yopen, test_size=0.25, random_state=51)

In [23]:
#Combine with the churn cases in train set
train6close1 = train6close[['Memberid', 'Age', 'NumberofSavingsProducts', 'CountofLoans', 'DBINDICATOR', 'CCINDICATOR', 'INDIRECT',
               'RecencyGap', 'Freq', 'DDAmt', 'LoanPayAmt', 'TM', 'NofDD_mean', 'NofBP_mean', 'NofDC_mean', 'NofIB_mean',
               'FC_mean', 'SofFC_mean', 'NofTT_mean', 'OL_mean', 'NofLP_mean', 'NofDD_sd', 'NofBP_sd', 'NofDC_sd', 'NofIB_sd',
               'FC_sd', 'SofFC_sd', 'NofTT_sd', 'OL_sd', 'NofLP_sd', 'NofDD_max', 'NofBP_max', 'NofDC_max', 'NofIB_max',
               'FC_max', 'SofFC_max', 'NofTT_max', 'OL_max', 'NofLP_max', 'NofDD_min', 'NofBP_min', 'NofDC_min', 'NofIB_min',
               'FC_min', 'SofFC_min', 'NofTT_min', 'OL_min', 'NofLP_min', 'NofDD_sk', 'NofBP_sk', 'NofDC_sk', 'NofIB_sk',
               'FC_sk', 'SofFC_sk', 'NofTT_sk', 'OL_sk', 'NofLP_sk', 'NofDD_k', 'NofBP_k', 'NofDC_k', 'NofIB_k',
               'FC_k', 'SofFC_k', 'NofTT_k', 'OL_k', 'NofLP_k']] #Extract input features for churn cases
X_train = pd.concat([X_train, train6close1], ignore_index=True)
X_train = X_train.sort_values(by=['Memberid'])
train6close1y = train6close[['Memberid', 'Status_Closed']]
y_train = pd.concat([y_train, train6close1y], ignore_index=True)
y_train = y_train.sort_values(by=['Memberid'])
y_train['Status_Closed'].value_counts() 

0    17972
1      666
Name: Status_Closed, dtype: int64

In [24]:
#Combine new cases in test set
Test2 = Test[['Memberid', 'Age', 'NumberofSavingsProducts', 'CountofLoans', 'DBINDICATOR', 'CCINDICATOR', 'INDIRECT',
               'RecencyGap', 'Freq', 'DDAmt', 'LoanPayAmt', 'TM', 'NofDD_mean', 'NofBP_mean', 'NofDC_mean', 'NofIB_mean',
               'FC_mean', 'SofFC_mean', 'NofTT_mean', 'OL_mean', 'NofLP_mean', 'NofDD_sd', 'NofBP_sd', 'NofDC_sd', 'NofIB_sd',
               'FC_sd', 'SofFC_sd', 'NofTT_sd', 'OL_sd', 'NofLP_sd', 'NofDD_max', 'NofBP_max', 'NofDC_max', 'NofIB_max',
               'FC_max', 'SofFC_max', 'NofTT_max', 'OL_max', 'NofLP_max', 'NofDD_min', 'NofBP_min', 'NofDC_min', 'NofIB_min',
               'FC_min', 'SofFC_min', 'NofTT_min', 'OL_min', 'NofLP_min', 'NofDD_sk', 'NofBP_sk', 'NofDC_sk', 'NofIB_sk',
               'FC_sk', 'SofFC_sk', 'NofTT_sk', 'OL_sk', 'NofLP_sk', 'NofDD_k', 'NofBP_k', 'NofDC_k', 'NofIB_k',
               'FC_k', 'SofFC_k', 'NofTT_k', 'OL_k', 'NofLP_k']] #Extract input features for new cases in testing period
X_test = pd.concat([X_test, Test2], ignore_index=True)
X_test = X_test.sort_values(by=['Memberid'])
Test2y = Test[['Memberid', 'Status_Closed']]
y_test = pd.concat([y_test, Test2y], ignore_index=True)
y_test = y_test.sort_values(by=['Memberid'])
y_test['Status_Closed'].value_counts()

0    9909
1     841
Name: Status_Closed, dtype: int64

In [25]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)